## Imports


In [ ]:

import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.stats.multicomp as multi

## Preparation

In [ ]:
data = pd.read_csv('../AIRPOL_data.csv', sep=';')

#Converter para float
data['Air_Pollution_Average[ug/m3]'] = pd.to_numeric(data['Air_Pollution_Average[ug/m3]'].str.replace(',', '.'), errors='coerce')

# Foi utilizado uma significância de 0.05
significancia = 0.05

# Data
data_spain = data[data['Country'] == 'Spain']
data_france = data[data['Country'] == 'France']
data_pt = data[data['Country'] == 'Portugal']
data_alb = data[data['Country'] == 'Albania']

# Samples
sample_sp = data_spain.sample(n=20)
sample_fr = data_france.sample(n=20)
sample_pt = data_pt.sample(n=20)
sample_alb = data_alb.sample(n=20)

levels_sp = sample_sp['Air_Pollution_Average[ug/m3]'].dropna()
levels_fr = sample_fr['Air_Pollution_Average[ug/m3]'].dropna()
levels_alb = sample_alb['Air_Pollution_Average[ug/m3]'].dropna()
levels_pt = sample_pt['Air_Pollution_Average[ug/m3]'].dropna()


## Resolution

In [ ]:
# =============================================================================
# 4. ANOVA para comparar os níveis médios de poluição entre 4 países:
#    Portugal, Albânia, Espanha e França (amostras de 20 registos cada)
# =============================================================================

# Realizar o teste ANOVA (H0: as médias dos 4 países são iguais)
f_stat, p_anova = stats.f_oneway(levels_pt, levels_alb, levels_sp, levels_fr)
print("\nResultado da ANOVA para 4 países:")
print("F-statistic:", f_stat)
print("p-value:", p_anova)

# Se a ANOVA indicar diferenças significativas, executar análise post-hoc
if p_anova < significancia:
    # Combinar os dados em um único DataFrame para o teste post-hoc
    df_anova = pd.concat([
        pd.DataFrame({'Country': 'Portugal', 'Polution_Level': levels_pt}),
        pd.DataFrame({'Country': 'Albânia', 'Polution_Level': levels_alb}),
        pd.DataFrame({'Country': 'Espanha', 'Polution_Level': levels_sp}),
        pd.DataFrame({'Country': 'França', 'Polution_Level': levels_fr})
    ])
    tukey = pairwise_tukeyhsd(endog=df_anova['Polution_Level'],
                              groups=df_anova['Country'],
                              alpha=0.05)
    print("\nResultados do teste post-hoc:")
    print(tukey)
else:
    print("\nNão foram encontradas diferenças significativas entre os países (p >= 0.05).")